In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
file_dir = '/kaggle/input/learn-ai-bbc/'
train_file = file_dir + 'BBC News Train.csv'
test_file = file_dir + 'BBC News Test.csv'

submit_file = file_dir + 'BBC News Sample Solution.csv'
glove50d_file_path = '/kaggle/input/gloveweights/glove.6B.50d.txt'

In [ ]:
trainData = pd .read_csv(train_file)

In [ ]:
trainData.head()

In [ ]:
trainData.Category.value_counts()

In [ ]:
def changeLabel(s):
    if s == 'sport':
        return 0;
    if s == 'business':
        return 1;
    if s == 'politics':
        return 2;
    if s == 'entertainment':
        return 3;
    if s == 'tech':
        return 4;
    
    

In [ ]:
#print(changeLabel('business'))
trainData['Category'] = trainData['Category'].apply(lambda x : changeLabel(x) )


In [ ]:
trainData['Category'].value_counts()

In [ ]:
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [ ]:
sentences = trainData['Text'].values

y = tf.keras.utils.to_categorical(trainData['Category'].values)

sentences_train, sentences_test, y_train, y_test = train_test_split(
    sentences, y, test_size=0.20, random_state=1000)

In [ ]:
y_train.shape

In [ ]:
y_test.shape

In [ ]:
from keras.preprocessing.text import Tokenizer

In [ ]:
tokenizer = Tokenizer(num_words=5000)

tokenizer.fit_on_texts(sentences_train)

X_train = tokenizer.texts_to_sequences(sentences_train)
X_test = tokenizer.texts_to_sequences(sentences_test)

In [ ]:
print(X_train[:2])

In [ ]:
vocab_size = len(tokenizer.word_index) + 1 
print(vocab_size)

In [ ]:
s = 'ab cd efg jk'
print(len(s.split(' ')))

In [ ]:
text = trainData[['Text']]
lenList = []
#text.apply(lambda x : print(len(x) )) 
for s in text.Text.values :
    lenList.append(len(s.split(' ')))


In [ ]:
text.head()

In [ ]:
import seaborn as sns

In [ ]:
sns.distplot(lenList)

# length 

In [ ]:
lenList.sort()
lenList[:3]

In [ ]:
lenList.sort(reverse=True)
lenList[:3]

In [ ]:
maxlen = 3000

In [ ]:
from keras.preprocessing.sequence import pad_sequences

In [ ]:
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [ ]:
X_train[:1]

In [ ]:
#from keras.models import Sequential
#from keras import layers
import tensorflow as tf

m = tf.keras.models

layers = tf.keras.layers

In [ ]:
# vocab size is 1750 
# input_length is size of review text after tokenization and pad sequance
embedding_dim = 120
model = m.Sequential()
model.add(layers.Embedding(input_dim=vocab_size,
                           output_dim=embedding_dim,
                           input_length=maxlen))

# model.add(layers.Flatten())
#model.add(layers.GlobalMaxPool1D())

model.add(layers.LSTM(200, activation='relu')) 
#model.add(layers.Dense(1024, activation='relu')) # 1024 -- 94.3
#model.add(layers.Dropout(0.2)) # -ve impactt
#model.add(layers.Dense(512, activation='relu'))

model.add(layers.Dense(50, activation='relu'))
#model.add(layers.Dropout(0.2)) # -ve impact
model.add(layers.Dense(5, activation='softmax'))


model.summary()

In [ ]:
model.compile(optimizer= tf.keras.optimizers.Adam(lr=.0005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
history = model.fit(X_train, y_train,
                    epochs=20,verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size = 10)
# batch size 10 or 20

In [ ]:
import matplotlib.pyplot as plt 

def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, 'val_'+string])
    plt.show()


In [ ]:
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")

In [ ]:
embedding dim
maxlen
Learning rate of optimizer
.001


adam = 
model.compile(optimizer = tf.keras.optimizers.Adam(lr=.0005),
                   loss = 'categorical_crossentropy',
                metrics = ['accuracy'])

In [ ]:
def create_embedding_matrix(filepath, word_index, embedding_dim):
    
    vocab_size = len(word_index) + 1 
    # Adding again 1 because of reserved 0 index
    
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath) as file:
        
        for line in file:
            
            # Fetching word and weights of that word from pre-trained file
            word, *vector = line.split()
            
            # putting word only if it exist in text for which model is created
            if word in word_index:
                # Find token number of each word from Toekenzier
                idx = word_index[word]
                #print("{} {} ".format(word,idx))
                # Create matrix with toekn and its vector from pre-trained weights
                embedding_matrix[idx] = np.array(vector, dtype=np.float32)[:embedding_dim]
    return embedding_matrix

In [ ]:
embedding_dim = 50
embedding_matrix = create_embedding_matrix(glove50d_file_path,
                                           tokenizer.word_index, 
                                           embedding_dim)

In [ ]:
# vocab size is 1750 
# input_length is size of review text after tokenization and pad sequance
embedding_dim = 50
model1 = m.Sequential()
model1.add(layers.Embedding(input_dim=vocab_size,
                           output_dim=embedding_dim,
                           weights=[embedding_matrix],
                           trainable=True, 
                           input_length=maxlen))

# model.add(layers.Flatten())
model1.add(layers.GlobalMaxPool1D())


model1.add(layers.Dense(1024, activation='relu')) # 1024 -- 94.3
#model.add(layers.Dropout(0.2)) # -ve impactt
#model.add(layers.Dense(512, activation='relu'))

model1.add(layers.Dense(100, activation='relu'))
#model.add(layers.Dropout(0.2)) # -ve impact
model1.add(layers.Dense(5, activation='softmax'))


model1.summary()

In [ ]:
model1.compile(optimizer= tf.keras.optimizers.Adam(lr=.0005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
history = model1.fit(X_train, y_train,
                    epochs=40,verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size = 10)

In [ ]:
#embedding_matrix[0:2]

In [ ]:
testFile = pd.read_csv(test_file)

In [ ]:
testFile.head()